In [ ]:
# !pip install catboost
# !pip install xgboost
# !pip install lightgbm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, roc_auc_score


import json
import nltk
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize.punkt import PunktToken
from nltk.stem.snowball import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import lda
from sklearn.decomposition import TruncatedSVD


import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('train.tsv', delimiter='\t',encoding='utf-8')
data

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,http://techcrunch.com/2010/09/08/kno-raises-46...,8958,"{""title"":""Kno Raises 46 Million More To Build ...",computer_internet,0.651067,3.010526,0.474747,0.222222,0.191919,0.191919,...,1,1,38,0,2219,99,11,0.040404,0.071429,0
7391,http://www.uncoached.com/category/why-i-miss-c...,8895,"{""title"":""Why I Miss College "",""body"":""Mar 30 ...",culture_politics,0.14192,2.208054,0.483333,0.246667,0.036667,0.026667,...,1,1,34,0,5672,300,4,0.020000,0.109453,0
7392,http://eatthis.menshealth.com/slide/sweet-pota...,1191,"{""title"":""Sweet Potatoes Eat This Not That i'...",recreation,0.196273,2.000000,0.315789,0.171053,0.105263,0.052632,...,?,1,43,0,848,76,5,0.434211,0.117647,1
7393,http://naturallyella.com/,5612,"{""title"":""Naturally Ella "",""body"":"" "",""url"":""n...",arts_entertainment,0.617876,1.026316,0.210526,0.052632,0.000000,0.000000,...,1,0,37,1,386,38,0,0.026316,0.333333,1


In [3]:
def info_columns(data_):
    total_na = data_.isna().sum().sum()
    print("Dimensional:", data_.shape[0], "rows,", data_.shape[1], "columns")
    print("Total NA values: %d" %(total_na))
    print("%38s%10s%10s%10s" %("Column Name", "Data type", "#Distinct", "#NaN"))
    col_name = data_.columns
    dtype = data_.dtypes
    uniq = data_.nunique()
    
    for i in range(len(col_name)):
        print("%38s%10s%10s%10s" %(col_name[i], dtype[i], uniq[i], data_[col_name[i]].isna().sum()))

In [4]:
info_columns(data)

Dimensional: 7395 rows, 27 columns
Total NA values: 0
                           Column Name Data type #Distinct      #NaN
                                   url    object      7395         0
                                 urlid     int64      7395         0
                           boilerplate    object      7394         0
                      alchemy_category    object        14         0
                alchemy_category_score    object      4806         0
                           avglinksize   float64      5710         0
                     commonlinkratio_1   float64      4476         0
                     commonlinkratio_2   float64      4038         0
                     commonlinkratio_3   float64      3266         0
                     commonlinkratio_4   float64      2695         0
                     compression_ratio   float64      6453         0
                           embed_ratio   float64       366         0
                            framebased     int64 

In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
urlid,7395.0,5305.704665,3048.384114,1.000000,2688.500000,5304.000000,7946.500000,10566.000000
avglinksize,7395.0,2.761823,8.619793,0.000000,1.602062,2.088235,2.627451,363.000000
commonlinkratio_1,7395.0,0.468230,0.203133,0.000000,0.340370,0.481481,0.616604,1.000000
commonlinkratio_2,7395.0,0.214080,0.146743,0.000000,0.105263,0.202454,0.300000,1.000000
commonlinkratio_3,7395.0,0.092062,0.095978,0.000000,0.022222,0.068627,0.133333,0.980392
commonlinkratio_4,7395.0,0.049262,0.072629,0.000000,0.000000,0.022222,0.065065,0.980392
compression_ratio,7395.0,2.255103,5.704313,0.000000,0.442616,0.483680,0.578227,21.000000
embed_ratio,7395.0,-0.103750,0.306545,-1.000000,0.000000,0.000000,0.000000,0.250000
framebased,7395.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
frameTagRatio,7395.0,0.056423,0.041446,0.000000,0.028502,0.045775,0.073459,0.444444


In [6]:
def fill_new_front_page(data_):
    dic = {x: 0 for x in data_['website_type'].values}
    for x in dic.keys():
        if (data_[data_['news_front_page'] == 1]['website_type'].values == x).sum() > (data_[data_['news_front_page'] == 0]['website_type'].values == x).sum():
            dic[x] = 1
        elif (data_[data_['news_front_page'] == 1]['website_type'].values == x).sum() == (data_[data_['news_front_page'] == 0]['website_type'].values == x).sum():
            dic[x] = np.random.randint(2)
        else:
            dic[x] = 0
    
    for i in range(data_.shape[0]):
        if data_[i:i+1]['news_front_page'].values == -1:
            data_[i:i+1]['news_front_page'].replace(-1, dic.get(data_[i:i+1]['website_type'].values[0]), inplace= True)
    
    return data_

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
def new_link_pre(df_train):
    linktrain1 = set(df_train[df_train['label'] == 1]['linkwordscore'].values)
    linktrain0 = set(df_train[df_train['label'] == 0]['linkwordscore'].values)
    link = list(linktrain1.intersection(linktrain0))
    
    val_count1 = []
    val_count0 = []
    for x in link:
        ah = df_train[df_train['linkwordscore'] == x]['label'].value_counts()
        val_count0.append(ah[0])
        val_count1.append(ah[1])
        
    for x in list(linktrain0 - linktrain1):
        link.append(x)
        val_count0.append(df_train[df_train['linkwordscore'] == x]['label'].value_counts()[0])
        val_count1.append(0)
    for x in list(linktrain1 - linktrain0):
        link.append(x)
        val_count1.append(df_train[df_train['linkwordscore'] == x]['label'].value_counts()[1])
        val_count0.append(0)
        
    rel = []
#     ler = []
#     for x in range(len(link)):
#         if val_count0[x] > val_count1[x]:
#             prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
#             rel.append(sigmoid(-prob_1 - 2*(1-prob_1)))
# #             ler.append(0)
            
#         elif val_count0[x] < val_count1[x]:
#             prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
#             rel.append(sigmoid(prob_1*2 + (1-prob_1)))
# #             ler.append(1)
#         else:
#             prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
#             rel.append(sigmoid(prob_1 - (1-prob_1)))
#             ler.append(np.random.randint(2))
    for x in range(len(link)):
        prob_1 = val_count1[x] / (val_count0[x] + val_count1[x])
        if prob_1 <= 0.4:
            rel.append(sigmoid(-prob_1 - 2*(1-prob_1)))
            
        elif prob_1 >= 0.6:
            prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
            rel.append(sigmoid(prob_1*2 + (1-prob_1)))
            
        else:
            prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
#             rel.append(sigmoid(prob_1 - (1-prob_1)))
            rel.append(sigmoid(2*(prob_1 - (1-prob_1))))
    
#     linkpd = pd.DataFrame({'linkscore': link, 'label = 1': val_count1, 'label = 0': val_count0, 'result_prob': rel, 'result_quan': ler})
    linkpd = pd.DataFrame({'linkscore': link, 'label = 1': val_count1, 'label = 0': val_count0, 'result_prob': rel})

    relsprob = []
#     relsquan = []
    for x in df_train['linkwordscore'].values:
        relsprob.append(linkpd[linkpd['linkscore'] == x]['result_prob'].values[0])
#         relsquan.append(linkpd[linkpd['linkscore'] == x]['result_quan'].values[0])
    
#     return relsprob, relsquan
    return relsprob, linkpd

In [9]:
def lld_nol_pre(df_train):
    linktrain1 = set(df_train[df_train['lengthyLinkDomain'] == 1]['numberOfLinks'].values)
    linktrain0 = set(df_train[df_train['lengthyLinkDomain'] == 0]['numberOfLinks'].values)
    link = list(linktrain1.intersection(linktrain0))
    
    val_count1 = []
    val_count0 = []
    for x in link:
        ah = df_train[df_train['numberOfLinks'] == x]['lengthyLinkDomain'].value_counts()
        val_count0.append(ah[0])
        val_count1.append(ah[1])
        
    for x in list(linktrain0 - linktrain1):
        link.append(x)
        val_count0.append(df_train[df_train['numberOfLinks'] == x]['lengthyLinkDomain'].value_counts()[0])
        val_count1.append(0)
    for x in list(linktrain1 - linktrain0):
        link.append(x)
        val_count1.append(df_train[df_train['numberOfLinks'] == x]['lengthyLinkDomain'].value_counts()[1])
        val_count0.append(0)
        
    rel = []
    for x in range(len(link)):
        prob_1 = val_count1[x] / (val_count0[x] + val_count1[x])
        if prob_1 <= 0.4:
            rel.append(sigmoid(-prob_1 - 2*(1-prob_1)))
            
        elif prob_1 >= 0.6:
            prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
            rel.append(sigmoid(prob_1*2 + (1-prob_1)))
            
        else:
            prob_1 = val_count1[x] / (val_count0[x] + val_count1[x]) 
            rel.append(sigmoid(2*(prob_1 - (1-prob_1))))
    
    linkpd = pd.DataFrame({'numberOfLinks': link, 'lengthyLinkDomain = 1': val_count1, 'lengthyLinkDomain = 0': val_count0, 'result_prob': rel})

    relsprob = []
    for x in df_train['numberOfLinks'].values:
        relsprob.append(linkpd[linkpd['numberOfLinks'] == x]['result_prob'].values[0])
    
    return relsprob

In [10]:
def featurizes(data_):
    text = data_['url'].str.split('//',n = -1, expand = True)[1]

    text2 = []
    text3 = []
    for x in text:
        spli = x.split('/')
        text2.append(spli[0])
        text3.append(x.split(spli[0] + '/')[1])

    subdomain = []
    webname = []
    domain = []
    for x in text2:
        spli = x.split('.')
        if len(spli) == 2:
            subdomain.append('')
            webname.append(spli[0])
            domain.append(spli[1])
        if len(spli) == 3:
            subdomain.append(spli[0])
            webname.append(spli[1])
            domain.append(spli[2])
        if len(spli) == 4:
            subdomain.append(spli[0])
            webname.append(spli[1])
            domain.append(spli[2] + '.' + spli[3])    
        if len(spli) > 4:
            subdomain.append(spli[0])
            webname.append(spli[1])
            domain.append(spli[2] + '.' + spli[3] + '.' + spli[4])

    text4 = []
    text5 = []
    for x in text3:
        spli = x.split('/')
        if len(spli) == 1:
            text4.append(spli[0])
            text5.append('')
        else:
            text4.append(spli[0])
            text5.append(spli[1])   

    punctuation = ['.', '?', '!', '=', '-', '_', '%', ':']
    for i in range(len(text4)):
        for j in punctuation:
            text4[i] = text4[i].replace(j, ' ')
            text5[i] = text5[i].replace(j, ' ')

    website_type = []
    website_type2 = [-1 for i in range(len(text5))]
    for i, x in enumerate(text4):
        spli = x.split(' ')
        if len(spli) == 1:
            website_type.append(spli[0])
        else:
            website_type.append(spli[0])
            website_type2[i] = spli[1]

    for i, x in enumerate(text5):
        if x == '':
            website_type2[i] = ''
        if website_type2[i] == -1:
            website_type2[i] = x.split(' ')[0]  
    
#     rels_prob, rels_quan = new_link_pre(data_)
    
    # Features from URL
    data_['subdomain'] = subdomain
    data_['webname'] = webname
    data_['domain'] = domain
    data_['website_type'] = website_type
    data_['website_type'] = data_['website_type'].replace({'2006':'YEAR', '2007':'YEAR', '2008':'YEAR',
                                '2009':'YEAR','2010':'YEAR', '2011':'YEAR', '2012':'YEAR','2013':'YEAR',
                            '01':'MONTH', '02':'MONTH','03':'MONTH','04':'MONTH','05':'MONTH','06':'MONTH',
                            '07':'MONTH','08':'MONTH','09':'MONTH','10':'MONTH','11':'MONTH','12':'MONTH'})
    
    data_['website_type2'] = website_type2
    data_['website_type2'] = data_['website_type2'].replace({'2006':'YEAR', '2007':'YEAR', '2008':'YEAR', 
                              '2009':'YEAR', '2010':'YEAR','2011':'YEAR', '2012':'YEAR','2013':'YEAR',
                            '01':'MONTH', '02':'MONTH','03':'MONTH','04':'MONTH','05':'MONTH','06':'MONTH',
                            '07':'MONTH','08':'MONTH','09':'MONTH','10':'MONTH','11':'MONTH','12':'MONTH',})
    
#     rels_prob = new_link_pre(data_)
#     data_['linkscorepredprob'] = rels_prob
#     data_['linkscorepredquan'] = rels_quan

    data_['wordInHLT'] = data_['non_markup_alphanum_characters'] * data_['linkwordscore'] / 100
    data_['wordError'] = data_['non_markup_alphanum_characters'] * data_['spelling_errors_ratio']
    
    lld = lld_nol_pre(data_)
    data_['lld_nol_prob'] = lld
    
    # Other Features
    data_['alchemy_category_score'] = pd.to_numeric(data_['alchemy_category_score'], errors= 'coerce')
    data_['alchemy_category_score'].fillna(data_['alchemy_category_score'].mean(), inplace= True)
    
#     data_['is_news'] = pd.to_numeric(data_['is_news'], errors= 'coerce')
#     data_['is_news'].fillna(0, inplace= True)
    
    data_['news_front_page'] = pd.to_numeric(data_['news_front_page'], errors= 'coerce')
    data_['news_front_page'].fillna(-1, inplace= True)
    
    data_.drop(columns= ['framebased','url', 'urlid', 'boilerplate', 'is_news'], inplace= True)
    
    # Encoder
    le = LabelEncoder()
    fea = ['alchemy_category', 'subdomain', 'webname', 'domain', 'website_type', 'website_type2']
    for x in fea:
        le.fit(data_[x])
        data_[x] = le.transform(data_[x])
    
    data_ = fill_new_front_page(data_)
    return data_

In [11]:
def fea_importance(new_data, model):
    tes = new_data.columns.tolist()
    tes.remove("label")
    feature_importance_df = pd.DataFrame()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["Feature"] = tes
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    cols = (feature_importance_df[["Feature", "importance"]]
            .groupby("Feature")
            .mean()
            .sort_values(by="importance", ascending=False)[:5000].index)

    best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

    plt.figure(figsize=(12,8))
    sns.barplot(x="importance",
                y="Feature",
                data=best_features.sort_values(by="importance",
                                                ascending=False))
    plt.title(model)
    plt.tight_layout()
    plt.show()

    imp = best_features.groupby('Feature')['importance'].median().sort_values(ascending=False)

In [12]:
new_data = data.copy()
new_data = featurizes(new_data)

In [13]:
rels_prob, linkpd = new_link_pre(new_data)
new_data['linkscorepredprob'] = rels_prob

In [14]:
# categorical = [var for var in new_data.columns if new_data[var].dtype == 'object']

# for var in categorical:
#   print(new_data[var].value_counts())

In [14]:
info_columns(new_data)

Dimensional: 7395 rows, 31 columns
Total NA values: 0
                           Column Name Data type #Distinct      #NaN
                      alchemy_category     int64        14         0
                alchemy_category_score   float64      4806         0
                           avglinksize   float64      5710         0
                     commonlinkratio_1   float64      4476         0
                     commonlinkratio_2   float64      4038         0
                     commonlinkratio_3   float64      3266         0
                     commonlinkratio_4   float64      2695         0
                     compression_ratio   float64      6453         0
                           embed_ratio   float64       366         0
                         frameTagRatio   float64      5911         0
                         hasDomainLink     int64         2         0
                            html_ratio   float64      7376         0
                           image_ratio   float64 

In [15]:
new_data.describe().T

,count,mean,std,min,25%,50%,75%,max
alchemy_category,7395.0,3.552130,3.711401,0.000000,0.000000,2.000000,8.000000,13.000000
alchemy_category_score,7395.0,0.603334,0.175952,0.070833,0.538758,0.603334,0.708279,0.999426
avglinksize,7395.0,2.761823,8.619793,0.000000,1.602062,2.088235,2.627451,363.000000
commonlinkratio_1,7395.0,0.468230,0.203133,0.000000,0.340370,0.481481,0.616604,1.000000
commonlinkratio_2,7395.0,0.214080,0.146743,0.000000,0.105263,0.202454,0.300000,1.000000
commonlinkratio_3,7395.0,0.092062,0.095978,0.000000,0.022222,0.068627,0.133333,0.980392
commonlinkratio_4,7395.0,0.049262,0.072629,0.000000,0.000000,0.022222,0.065065,0.980392
compression_ratio,7395.0,2.255103,5.704313,0.000000,0.442616,0.483680,0.578227,21.000000
embed_ratio,7395.0,-0.103750,0.306545,-1.000000,0.000000,0.000000,0.000000,0.250000
frameTagRatio,7395.0,0.056423,0.041446,0.000000,0.028502,0.045775,0.073459,0.444444


In [20]:
stop_words = stopwords.words('english')
stop_word_add = ['becau','abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv']
for i in stop_word_add:
    stop_words.append(i)
    
def preprocessor(text):
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P).,!*-"', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

stemmer = SnowballStemmer("english")
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = PunktToken().tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [19]:
def tfidf(data_):
    tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                          token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1, smooth_idf=1,sublinear_tf=1,
                          stop_words= stop_words, tokenizer= tokenizer_porter, preprocessor= preprocessor)
    tfv.fit(data_['body'])
    data_['body'] = tfv.transform(data_['body'])
    return data_

In [20]:
def documentFrequencyVector(trainData):
    tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                          token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1, smooth_idf=1,sublinear_tf=1,
                          stop_words= stop_words, tokenizer= tokenizer_porter, preprocessor= preprocessor)
    tfv.fit(trainData)
    X = tfv.transform(trainData)
    return X

In [21]:
# LDA parameters
numTopics = 1000
numIterations = 50

def transformInLDATopics(trainData):
    trainData = documentFrequencyVector(trainData)
    model = lda.LDA(n_topics=numTopics, n_iter=numIterations, random_state=1)
    model.fit(trainData)
    doc_topic = model.doc_topic_
    return doc_topic

def getLdaTopicsVector(trainData):
    doc_topic = transformInLDATopics(trainData)
    topicVector = []
    for i in range(X.shape[0]):
        topicVector.append(doc_topic[i].argmax())
    return topicVector

In [17]:
main_data = data[['boilerplate', 'label']]

boilerplate = main_data['boilerplate']

dicts = {
    'title' : [],
    'body' : [],
    'url' : []
}
for i in range(len(boilerplate)):
    temp = json.loads(boilerplate[i])
    for j in dicts.keys():
        dicts[j].append(temp.get(j, None))
        
text_data = pd.DataFrame()
text_data['title'] = dicts['title']
text_data['body'] = dicts['body']
text_data['url'] = dicts['url']

In [18]:
def fillNanAndNone(data_):
    fea = ['title', 'body', 'url']
    for name in fea:
        data_[name].fillna(" ", inplace= True)
    return data_

text_data = fillNanAndNone(text_data)

In [21]:
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word', sublinear_tf=True,
                token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf= True, smooth_idf= True, binary=False,
                stop_words= stop_words, tokenizer= tokenizer_porter, preprocessor= preprocessor)

bodyVect = tfv.fit_transform(text_data['body'])
titleVect = tfv.fit_transform(text_data['title'])

In [22]:
alchemyCategories_df = pd.get_dummies(data.alchemy_category, prefix = 'DUM')
alchemyCategories_df

,DUM_?,DUM_arts_entertainment,DUM_business,DUM_computer_internet,DUM_culture_politics,DUM_gaming,DUM_health,DUM_law_crime,DUM_recreation,DUM_religion,DUM_science_technology,DUM_sports,DUM_unknown,DUM_weather
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7391,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7392,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7393,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
titleDecomposed = TruncatedSVD(random_state = 123, n_components = 10).fit_transform(titleVect)
bodyDecomposed = TruncatedSVD(random_state = 123, n_components = 50).fit_transform(bodyVect)

textDecomposed = np.hstack([titleDecomposed,bodyDecomposed])

In [24]:
dataTrain = new_data[[x for x in new_data.columns if x != 'label']]
dataTrain

,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,frameTagRatio,...,spelling_errors_ratio,subdomain,webname,domain,website_type,website_type2,wordInHLT,wordError,lld_nol_prob,linkscorepredprob
0,2,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0.090774,...,0.079130,404,321,18,1145,612,1301.76,429.198815,0.855422,0.553639
1,8,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0.098707,...,0.125448,404,2015,18,1529,666,1989.20,623.853048,0.862158,0.448460
2,6,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0.072448,...,0.057613,404,1614,18,888,1028,1232.00,129.053499,0.860348,0.163614
3,6,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0.095861,...,0.100858,404,705,18,337,527,656.88,276.049356,0.839185,0.553639
4,11,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0.024908,...,0.082569,0,307,18,395,60,1684.48,993.467886,0.867036,0.581679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,3,0.651067,3.010526,0.474747,0.222222,0.191919,0.191919,0.474273,0.0,0.177043,...,0.071429,0,2441,18,337,527,843.22,158.499999,0.582570,0.492538
7391,4,0.141920,2.208054,0.483333,0.246667,0.036667,0.026667,0.558184,0.0,0.057377,...,0.109453,404,2732,18,524,1743,1928.48,620.815919,0.880797,0.445759
7392,8,0.196273,2.000000,0.315789,0.171053,0.105263,0.052632,0.692529,0.0,0.124122,...,0.117647,99,1614,18,1433,1624,364.64,99.764706,0.835019,0.500000
7393,1,0.617876,1.026316,0.210526,0.052632,0.000000,0.000000,21.000000,-1.0,0.097778,...,0.333333,0,1749,18,0,0,142.82,128.666667,0.149912,0.409607


In [90]:
yt = new_data.label
Xt = np.concatenate((dataTrain, alchemy_dum_alt[data_test.shape[0]:].values), 1)
X_alt = np.hstack([Xt, textDecomposed])
X_train, X_test, y_train, y_test = train_test_split(X_alt, yt, test_size=0.2, random_state=102)

print('Training set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

Training set: (5916, 104) (5916,)
Test set: (1479, 104) (1479,)


In [63]:
alchemy_dum_alt[:data.shape[0]].shape

(7395, 14)

In [26]:
base_classifiers = [
                    ('Random Forest', RandomForestClassifier(n_estimators=1000, n_jobs= 4, min_samples_leaf= 2, bootstrap= True, min_samples_split= 5, max_features= .1, max_samples= 30)),
                    ('AdaBoost Classifier', AdaBoostClassifier()),
                    ('Gradient Boosting Classifier', GradientBoostingClassifier(n_estimators= 300, learning_rate= 0.08, max_features= 4, min_samples_leaf= 2, min_samples_split= 5, max_depth= 7)),
                    ('Extra Trees Classifier', ExtraTreesClassifier()),
                    ('LightGBM', LGBMClassifier(n_estimators= 300, learning_rate= 0.08, num_leaves= 50,boosting_type= 'gbdt', objective= 'binary', sub_feature= 0.5, max_depth= 7)),
                    ('XGBM', XGBClassifier())]

# ('KNN', KNeighborsClassifier()),
# ('Naive Bayes', GaussianNB()),
# ('Decision Tree', DecisionTreeClassifier()),

In [85]:
pred_prob = []
for x in range(3):
    clf = StackingClassifier(estimators= base_classifiers, final_estimator= LogisticRegression())
    clf.fit(X_train, y_train)
    pred_prob.append(clf.predict_proba(X_test)[:, 1])

[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[00:38:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[00:41:19] WARNING: ../src/learner.cc:1061:

In [27]:
def voting_prob(lis):
    return np.mean(lis)

In [86]:
lis = [[] for i in range(y_test.shape[0])]
for i in range(y_test.shape[0]):
    for j in range(len(pred_prob)):
            lis[i].append(pred_prob[j][i])
            
rel = []
for i in range(y_test.shape[0]):
    rel.append(voting_prob(lis[i]))
    
print("Roc_auc_accu", roc_auc_score(y_test, rel) * 100)

Roc_auc_accu 86.07912984961087


In [88]:
def crossVal(clf, X_data, Y_data, NFOLDS = 10): 
    import sklearn.ensemble as ens
    from sklearn import metrics
    from sklearn.cross_validation import StratifiedKFold

    skf = StratifiedKFold(Y_data, n_folds = NFOLDS)
    auc = []
    for train_index, test_index in skf:
        x_train, x_test = X_data[train_index], X_data[test_index]
        y_train, y_test = Y_data[train_index], Y_data[test_index]
        
        clf.fit(x_train, y_train)
        
        try: 
            probs = clf.predict_proba(x_test)
            y_hat = probs[:,1]
        except:
            y_hat = clf.predict(x_test)
        auc.append(metrics.roc_auc_score(y_test, y_hat))
    print(auc)
    print(np.mean(auc))
    return auc

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters3 = {'learning_rate': [0.05, 0.1, 0.25, 0.5, 0.75],
              'n_estimators' : [100, 300, 500, 700, 1000, 2000, 4000],
              'max_depth' : [2,3,4,5,10,15,20,50,100],
              'min_samples_split' : [2,4,6,8,10],
              'min_samples_leaf' : [1,2,5],
              'subsample' : [0.5, 0.75, 1]}
clfGBMGrid = GridSearchCV(GradientBoostingClassifier(random_state = 123), parameters3).fit(X_train,y_train)

In [28]:
data_test = pd.read_csv('test.tsv', sep ='\t')
new_data_test = data_test.copy()
new_data_test = featurizes(new_data_test)

In [29]:
prob = []
for x in new_data_test['linkwordscore'].values:
    prob.append(linkpd[linkpd['linkscore'] == x]['result_prob'].values[0])
    
new_data_test['linkscorepredprob'] = prob

In [40]:
main_data_test = data_test[['boilerplate']]

boilerplate_test = main_data_test['boilerplate']

dicts_test = {
    'title' : [],
    'body' : [],
    'url' : []
}
for i in range(len(boilerplate_test)):
    temp = json.loads(boilerplate_test[i])
    for j in dicts_test.keys():
        dicts_test[j].append(temp.get(j, None))
        
text_data_test = pd.DataFrame()
text_data_test['title'] = dicts_test['title']
text_data_test['body'] = dicts_test['body']
text_data_test['url'] = dicts_test['url']

text_data_test = fillNanAndNone(text_data_test)

In [42]:
bodyVect_test = tfv.fit_transform(text_data_test['body'])
titleVect_test = tfv.fit_transform(text_data_test['title'])

In [33]:
alchemyCategories_df_test = pd.get_dummies(data_test.alchemy_category, prefix = 'DUM')
alchemyCategories_df_test

,DUM_?,DUM_arts_entertainment,DUM_business,DUM_computer_internet,DUM_culture_politics,DUM_gaming,DUM_health,DUM_law_crime,DUM_recreation,DUM_religion,DUM_science_technology,DUM_sports
0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3166,1,0,0,0,0,0,0,0,0,0,0,0
3167,1,0,0,0,0,0,0,0,0,0,0,0
3168,1,0,0,0,0,0,0,0,0,0,0,0
3169,0,0,0,0,0,0,0,0,0,0,0,1


In [65]:
alchemy_dum_alt[data.shape[0]:]

,DUM_?,DUM_arts_entertainment,DUM_business,DUM_computer_internet,DUM_culture_politics,DUM_gaming,DUM_health,DUM_law_crime,DUM_recreation,DUM_religion,DUM_science_technology,DUM_sports,DUM_unknown,DUM_weather
4224,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4225,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4226,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4227,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7391,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7392,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7393,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
titleDecomposed_test = TruncatedSVD(random_state = 123, n_components = 10).fit_transform(titleVect_test)
bodyDecomposed_test = TruncatedSVD(random_state = 123, n_components = 50).fit_transform(bodyVect_test)

textDecomposed_test = np.hstack([titleDecomposed_test,bodyDecomposed_test])

In [91]:
Xt_test = np.concatenate((new_data_test, alchemy_dum_alt[:data_test.shape[0]].values), 1)
X_alt_test = np.hstack([Xt_test, textDecomposed_test])

In [61]:
alchemy_alt = data_test['alchemy_category'].append(data['alchemy_category'])
alchemy_dum_alt = pd.get_dummies(alchemy_alt, prefix = 'DUM')

In [92]:
pred_prob = []
for x in range(3):
    clf = StackingClassifier(estimators= base_classifiers, final_estimator= LogisticRegression())
    clf.fit(X_alt, yt)
    pred_prob.append(clf.predict_proba(X_alt_test)[:, 1])

[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[10:04:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[10:05:53] WARNING: ../src/learner.cc:1061:

In [93]:
lis = [[] for i in range(data_test.shape[0])]
for i in range(data_test.shape[0]):
    for j in range(len(pred_prob)):
            lis[i].append(pred_prob[j][i])
            
rel = []
for i in range(data_test.shape[0]):
    rel.append(voting_prob(lis[i]))

In [95]:
rel

[0.7565313952715192,
 0.718988217317968,
 0.3458959415760187,
 0.786802517202252,
 0.6809289039709885,
 0.7962149366847516,
 0.7343741286385095,
 0.8067536786418253,
 0.718422891074363,
 0.514851667832774,
 0.6314197667051171,
 0.46167208517634756,
 0.5919986424676389,
 0.514329512099625,
 0.7678246494768045,
 0.8405749508618611,
 0.6261431095849411,
 0.7077582016255741,
 0.8469117871563765,
 0.6744440234454987,
 0.5171634993629045,
 0.7189617174444235,
 0.689566744948185,
 0.7325683451118866,
 0.7098949997434737,
 0.35659358301614175,
 0.785142387239246,
 0.6240367378641202,
 0.6970943429315309,
 0.7105404543956011,
 0.6579577678251297,
 0.7921519851093817,
 0.7073734744785046,
 0.7636973563094624,
 0.8327600263866479,
 0.7608682201521165,
 0.7991737191377389,
 0.8896509908212908,
 0.7491848846761772,
 0.6727441588079307,
 0.6037643253216389,
 0.7543270909948959,
 0.19403140437610444,
 0.6849357815241778,
 0.7943331285559944,
 0.8252861326169217,
 0.876433989255815,
 0.791747885555083

In [94]:
xx = pd.DataFrame({'urlid': data_test['urlid'],'label': rel})
compression_opts = dict(method='zip',
                        archive_name='out123.csv')  
xx.to_csv('he.zip', index=False,
          compression=compression_opts)  

In [4]:
import json
from collections import Counter
from nltk.corpus import stopwords
import re 
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
main_data = data[['boilerplate', 'label']]

In [6]:
boilerplate = main_data['boilerplate']

dicts = {
    'title' : [],
    'body' : [],
    'url' : []
}
for i in range(len(boilerplate)):
    temp = json.loads(boilerplate[i])
    for j in dicts.keys():
        dicts[j].append(temp.get(j, None))

In [7]:
text_data = pd.DataFrame()
text_data['title'] = dicts['title']
text_data['body'] = dicts['body']
text_data['url'] = dicts['url']

text_data

,title,body,url
0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...,bloomberg news 2010 12 23 ibm predicts hologra...
1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...,popsci technology article 2012 07 electronic f...
2,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...,menshealth health flu fighting fruits cm mmc F...
3,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...,dumblittleman 2007 12 10 foolproof tips for be...
4,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...,bleacherreport articles 1205138 the 50 coolest...
...,...,...,...
7390,Kno Raises 46 Million More To Build Most Power...,Marc Andreessen is normally enthusiastic about...,techcrunch 2010 09 08 kno raises 46 million mo...
7391,Why I Miss College,Mar 30 2009 I d like to congratulate Jane on h...,uncoached category why i miss college
7392,Sweet Potatoes Eat This Not That i'm eating t...,They re loaded with vitamin C which smoothes o...,eatthis menshealth slide sweet potatoes slides...
7393,Naturally Ella,,naturallyella


In [8]:
ano_text_data = text_data.copy()

In [31]:
def fillNanAndNone(data_):
    fea = ['title', 'body', 'url']
    for name in fea:
        data_[name].fillna(" ", inplace= True)
#         for i, text in enumerate(data_[name]):
#             if len(text) == 1:
#                 data_[name][i] = "-1"
    return data_

ano_text_data = fillNanAndNone(ano_text_data)

In [32]:
ano_text_data.head(60)

,title,body,url
0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...,bloomberg news 2010 12 23 ibm predicts hologra...
1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...,popsci technology article 2012 07 electronic f...
2,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...,menshealth health flu fighting fruits cm mmc F...
3,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...,dumblittleman 2007 12 10 foolproof tips for be...
4,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...,bleacherreport articles 1205138 the 50 coolest...
5,Genital Herpes Treatment,Genital herpes is caused by herpes simplex vir...,conveniencemedical genital herpes home php
6,fashion lane American Wild Child,Our favorite summer holiday is just around the...,gofashionlane blogspot tw 2012 06 american wil...
7,Racing For Recovery by Dean Johnson racing for...,Racing For Recovery is the growing idea that d...,insidershealth article racing for recovery 3471
8,Valet The Handbook 31 Days 31 days,Resolutions are for suckers Instead of swearin...,valetmag the handbook features 31 days index p...
9,Cookies and Cream Brownies How Sweet It Is,More brownies It seems that I can t get throug...,howsweeteats 2010 03 24 cookies and cream brow...


In [10]:
def val_count_stopwords(data_):
    fea = ['title', 'body', 'url']
    stop_words = stopwords.words('english')
    for name in fea:
        vocab = Counter()
        count = 0
        for twit in data_[name]:
            for word in twit.split(' '):
                vocab[word] += 1
            count += 1
        print(name, vocab.most_common(20), end= 'aaa\n\n')

        vocab_reduced = Counter()
        for w, c in vocab.items():
            if not w in stop_words:
                vocab_reduced[w]=c

        print(name, vocab_reduced.most_common(20), end= 'bbb\n\n')
val_count_stopwords(ano_text_data)

AttributeError: 'NoneType' object has no attribute 'split'

In [13]:

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text
print(preprocessor('I like it :), |||<><>'))

i like it  :)


In [14]:

porter = PorterStemmer()

# Split a text into list of words
def tokenizer(text):
    return text.split()

# Split a text into list of words and apply stemming technic
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [17]:
stop_words = stopwords.words('english')

In [34]:
A = ano_text_data['body']
b = main_data['label']
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=102)
A_train.shape, A_test.shape, b_train.shape, b_test.shape

((5916,), (1479,), (5916,), (1479,))

In [35]:


tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(A_train, b_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x7f49d0f5bd30>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x7f49d0f5b700>)),
                ('clf', LogisticRegression(random_state=0))])

In [36]:

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(A_test)
print('accuracy:',accuracy_score(b_test,predictions))
print('confusion matrix:\n',confusion_matrix(b_test,predictions))

accuracy: 0.7809330628803245
confusion matrix:
 [[645  98]
 [226 510]]


In [37]:
print("Roc_auc_accu", roc_auc_score(b_test, clf.predict_proba(A_test)[:, 1]) * 100)

Roc_auc_accu 85.49478099947333
